In [1]:
import dask
import dask.dataframe as dd
from dask.distributed import Client
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
import numpy as np
import seaborn as sns
import category_encoders as ce
from numba import njit
from sklearn.pipeline import make_pipeline

In [2]:
client = Client(n_workers = 3)

In [3]:
client

Client Scheduler: tcp://127.0.0.1:59488 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 3 Cores: 6 Memory: 17.11 GB


In [4]:
%cd ds1-predictive-modeling-challenge

E:\ds\DS-Unit-3-Sprint-3-Big-Data\module1-aws-sagemaker\ds1-predictive-modeling-challenge


In [5]:
x = dd.read_csv('train_features.csv')
y = dd.read_csv('train_labels.csv')
y = y.drop(["id"],axis=1)
y = y.mask(y == "functional needs repair", "non functional")

In [6]:
len(x), len(y)

(59400, 59400)

In [7]:
x.columns.values

array(['id', 'amount_tsh', 'date_recorded', 'funder', 'gps_height',
       'installer', 'longitude', 'latitude', 'wpt_name', 'num_private',
       'basin', 'subvillage', 'region', 'region_code', 'district_code',
       'lga', 'ward', 'population', 'public_meeting', 'recorded_by',
       'scheme_management', 'scheme_name', 'permit', 'construction_year',
       'extraction_type', 'extraction_type_group',
       'extraction_type_class', 'management', 'management_group',
       'payment', 'payment_type', 'water_quality', 'quality_group',
       'quantity', 'quantity_group', 'source', 'source_type',
       'source_class', 'waterpoint_type', 'waterpoint_type_group'],
      dtype=object)

In [6]:
x_train, x_test = x.random_split([0.75,0.25],random_state =42)
y_train, y_test = y.random_split([0.75,0.25],random_state =42)

In [7]:
len(x_train),len(y_train)

(44659, 44659)

In [10]:
y_train.shape

(Delayed('int-3ebfc541-2bbc-4111-8da1-bf318dc84d53'), 1)

In [11]:
%%time
Kn = KNeighborsClassifier(n_neighbors=25,weights='distance')
Kn.fit(x_train[["latitude","longitude"]],y_train)
Kn.predict(x_train[["latitude","longitude"]])
Kn.score(x_test[["latitude","longitude"]],y_test.compute())

C:\Users\cwcol\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  


Wall time: 1.82 s


In [12]:
%%time
Kn = KNeighborsClassifier(n_neighbors=25,weights='distance', n_jobs=-1)
Kn.fit(x_train[["latitude","longitude"]],y_train)
Kn.predict(x_train[["latitude","longitude"]])
Kn.score(x_test[["latitude","longitude"]],y_test.compute())

C:\Users\cwcol\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  


Wall time: 1.88 s


In [13]:
Kn.score(x_test[["latitude","longitude"]],y_test.compute())

0.7115528118852181

In [14]:
z = x[["funder", "installer", "permit", "scheme_management","construction_year","payment","latitude","longitude",]]
z = z.mask(z == 0, np.nan).mask(z == "unknown",np.nan)
z = z.isna()
z["poorly_documented"] = False
for i in z.columns.values:
    z["poorly_documented"] = (z["poorly_documented"] | z[i])
x["poorly_documented"] = z["poorly_documented"]

In [15]:
x["construction_year"] = x["construction_year"].mask(x == 0,2000)

In [16]:
x.columns

Index(['id', 'amount_tsh', 'date_recorded', 'funder', 'gps_height',
       'installer', 'longitude', 'latitude', 'wpt_name', 'num_private',
       'basin', 'subvillage', 'region', 'region_code', 'district_code', 'lga',
       'ward', 'population', 'public_meeting', 'recorded_by',
       'scheme_management', 'scheme_name', 'permit', 'construction_year',
       'extraction_type', 'extraction_type_group', 'extraction_type_class',
       'management', 'management_group', 'payment', 'payment_type',
       'water_quality', 'quality_group', 'quantity', 'quantity_group',
       'source', 'source_type', 'source_class', 'waterpoint_type',
       'waterpoint_type_group', 'poorly_documented'],
      dtype='object')

In [17]:
x_train, x_test = x.random_split([0.75,0.25],random_state =42)
y_train, y_test = y.random_split([0.75,0.25],random_state =42)

In [25]:
c = ["basin","region","region_code","district_code","scheme_management","extraction_type","management",\
         "extraction_type_group","quantity_group", "payment", "waterpoint_type","water_quality","quality_group","public_meeting"\
        ,"lga","source","source_class","extraction_type_class","management_group",\
          'gps_height',"population","construction_year"]
cols1 = ["basin","region","region_code","district_code","scheme_management","extraction_type","management",\
         "extraction_type_group","quantity_group", "payment", "waterpoint_type","water_quality","quality_group",\
         "public_meeting","lga","poorly_documented","source","source_class","extraction_type_class", \
         "top_installer","management_group", "top_funder"]
class_weights = {'functional':1,'non functional':1.2,'functional needs repair':1}


In [26]:
from dask.distributed import Client
from sklearn.externals.joblib import parallel_backend

client = Client()  # Connect to a Dask Cluster

with parallel_backend('dask'):
    pipeline = make_pipeline(
    ce.OneHotEncoder(cols = c,use_cat_names=True),
    LogisticRegression(solver='lbfgs',class_weight=class_weights)
    )

C:\Users\cwcol\Anaconda3\lib\site-packages\distributed\bokeh\core.py:57: UserWarning: 
Port 8787 is already in use. 
Perhaps you already have a cluster running?
Hosting the diagnostics dashboard on a random port instead.
  warnings.warn('\n' + msg)


In [27]:
%%time
from sklearn.externals import joblib   
with joblib.parallel_backend('dask'):
    pipeline.fit(x_train[c].values,y_train.values)
    y_pred = pipeline.predict(x_test[c].values)
    accuracy_score(y_test,y_pred)

ValueError: Unexpected input type: <class 'dask.array.core.Array'>

In [20]:
x_train[c]

,basin,region,region_code,district_code,scheme_management,extraction_type,management,extraction_type_group,quantity_group,payment,waterpoint_type,water_quality,quality_group,public_meeting,lga,poorly_documented,source,source_class,extraction_type_class,management_group,gps_height,population,construction_year
npartitions=1,,,,,,,,,,,,,,,,,,,,,,,
,object,object,int64,int64,object,object,object,object,object,object,object,object,object,object,object,bool,object,object,object,object,int64,int64,int64
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...


In [21]:
y_train["status_group"].value_counts().compute()

functional        24178
non functional    20481
Name: status_group, dtype: int64

In [30]:
c = ["latitude","longitude","construction_year"]
x_train[c]

,latitude,longitude,construction_year
npartitions=1,,,
,float64,float64,int64
,...,...,...


In [31]:
x_train[c].isna().sum().compute()

latitude             0
longitude            0
construction_year    0
dtype: int64

In [42]:
x_train = x_train[c]

In [46]:
x_train["construction_year"] = x_train["construction_year"].astype("float")

In [50]:
x_train.compute()

,latitude,longitude,construction_year
0,-9.856322,34.938093,1999.0
1,-2.147466,34.698766,2010.0
2,-3.821329,37.460664,2009.0
3,-11.155298,38.486161,1986.0
4,-1.825359,31.130847,0.0
5,-4.765587,39.172796,2009.0
6,-3.766365,33.362410,0.0
8,-5.146712,32.711100,0.0
9,-1.257051,30.626991,0.0
10,-7.034139,39.209518,2011.0


In [51]:
from dask_ml.linear_model import LogisticRegression
client = Client()

lr = LogisticRegression()
lr.fit(x_train.values, y_train["status_group"].values)

C:\Users\cwcol\Anaconda3\lib\site-packages\distributed\bokeh\core.py:57: UserWarning: 
Port 8787 is already in use. 
Perhaps you already have a cluster running?
Hosting the diagnostics dashboard on a random port instead.
  warnings.warn('\n' + msg)


TypeError: can't multiply sequence by non-int of type 'float'

In [ ]:
np.dot(x_train.values,x_train.values.T)

In [20]:
from sklearn.datasets import make_classification

X, y = make_classification(n_samples=10000,
                           n_features=500,
                           n_classes=2,
                           n_redundant=250,
                           random_state=42)

from sklearn import linear_model, decomposition
from sklearn.pipeline import Pipeline

logistic = linear_model.LogisticRegression()
pca = decomposition.PCA()
pipe = Pipeline(steps=[('pca', pca),
                       ('logistic', logistic)])


#Parameters of pipelines can be set using ‘__’ separated parameter names:
grid = dict(pca__n_components=[50, 100, 150, 250],
            logistic__C=[1e-4, 1.0, 10, 1e4],
            logistic__penalty=['l1', 'l2'])



estimator = GridSearchCV(pipe, grid)

estimator.fit(X, y)


ModuleNotFoundError: No module named 'sklearn.grid_search'